In [132]:
import numpy as np
import pandas as pd
import json

In [133]:
with open('data.json', 'r') as f:
    labels = json.load(f)
labels

{'Boy': 0,
 'Can': 1,
 'Eat': 2,
 'Fine': 3,
 'Girl': 4,
 'Help': 5,
 'How': 6,
 'Hungry': 7,
 'I': 8,
 'Mother': 9,
 'Namaste': 10,
 'Name': 11,
 'Parents': 12,
 'Sister': 13,
 'Sleep': 14,
 'This': 15,
 'You': 16}

In [134]:
data=pd.read_csv('dataset.csv')
data.head()

,Class Label,X0001,Y0001,Z0001,X0101,Y0101,Z0101,X0201,Y0201,Z0201,...,Z6220,X6320,Y6320,Z6320,X6420,Y6420,Z6420,X6520,Y6520,Z6520
0,Boy,-0.290488,-75.818914,-3.229877,-0.166410,-77.328566,-3.061451,-0.098528,-82.594196,-2.995450,...,-0.789076,0.607418,0.957875,-0.403516,0.299876,0.551094,-0.700673,0.556823,0.893926,-0.055159
1,Boy,-1.142780,-504.405201,-11.642699,-1.142780,-504.405201,-11.642699,-1.142780,-504.405201,-11.642699,...,-0.303101,0.853242,0.551130,-0.508979,0.312054,0.981356,-0.225069,0.598794,0.899667,0.023955
2,Boy,-1.068371,130.701614,9.520099,-1.068371,130.701614,9.520099,-1.068371,130.701614,9.520099,...,-0.360270,0.853557,0.546488,-0.445940,0.312178,0.975818,-0.276978,0.599459,0.901850,0.024480
3,Boy,-0.229189,-728.344904,-10.610774,-0.111319,-760.483651,-9.444202,-0.050989,-814.690937,-8.885182,...,-1.450281,0.686321,0.983726,-0.696741,0.151945,0.553058,-1.307723,0.605042,0.889703,-0.100882
4,Boy,-0.876114,-24.004894,1.529031,-0.800043,-21.179765,1.634938,-0.749626,-17.375034,1.698469,...,-0.419257,0.596819,0.970013,-0.363273,0.386495,0.966971,-0.328736,0.544566,0.908291,-0.058046


In [135]:
Y = data['Class Label']
Y

0      Boy
1      Boy
2      Boy
3      Boy
4      Boy
      ... 
717    You
718    You
719    You
720    You
721    You
Name: Class Label, Length: 722, dtype: object

In [136]:
Y = Y.map(labels)
type(Y)

pandas.core.series.Series

In [137]:
# Y=pd.DataFrame(Y)
Y=np.array(Y)
Y

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  5,  5,
        5,  5,  5,  5,  5

In [138]:
# from tensorflow.keras.utils import to_categorical
# Y=to_categorical(Y).astype(int)
# Y

In [139]:
Y.shape

(722,)

In [140]:
data.drop(['Class Label'], axis=1, inplace=True)
data.head()

,X0001,Y0001,Z0001,X0101,Y0101,Z0101,X0201,Y0201,Z0201,X0301,...,Z6220,X6320,Y6320,Z6320,X6420,Y6420,Z6420,X6520,Y6520,Z6520
0,-0.290488,-75.818914,-3.229877,-0.166410,-77.328566,-3.061451,-0.098528,-82.594196,-2.995450,-0.065065,...,-0.789076,0.607418,0.957875,-0.403516,0.299876,0.551094,-0.700673,0.556823,0.893926,-0.055159
1,-1.142780,-504.405201,-11.642699,-1.142780,-504.405201,-11.642699,-1.142780,-504.405201,-11.642699,-1.142780,...,-0.303101,0.853242,0.551130,-0.508979,0.312054,0.981356,-0.225069,0.598794,0.899667,0.023955
2,-1.068371,130.701614,9.520099,-1.068371,130.701614,9.520099,-1.068371,130.701614,9.520099,-1.068371,...,-0.360270,0.853557,0.546488,-0.445940,0.312178,0.975818,-0.276978,0.599459,0.901850,0.024480
3,-0.229189,-728.344904,-10.610774,-0.111319,-760.483651,-9.444202,-0.050989,-814.690937,-8.885182,-0.018511,...,-1.450281,0.686321,0.983726,-0.696741,0.151945,0.553058,-1.307723,0.605042,0.889703,-0.100882
4,-0.876114,-24.004894,1.529031,-0.800043,-21.179765,1.634938,-0.749626,-17.375034,1.698469,-0.744764,...,-0.419257,0.596819,0.970013,-0.363273,0.386495,0.966971,-0.328736,0.544566,0.908291,-0.058046


In [141]:
data.shape

(722, 3960)

In [142]:
data.isna().sum()

X0001      0
Y0001      0
Z0001      0
X0101      0
Y0101      0
        ... 
Y6420    300
Z6420    300
X6520    300
Y6520    300
Z6520    300
Length: 3960, dtype: int64

In [143]:
# X=[]
for i in range(data.shape[0]):
    # for j in range(data.shape[1]):
    #     if data.iloc[i][j].isna():
    #         data.iloc[i][j] = np.mean(data.iloc[i])
    data.iloc[i].fillna(data.iloc[i].mean(), inplace=True)
    # X.append(np.array(data.iloc[0][1:]))

In [144]:
data.isna().sum()

X0001    0
Y0001    0
Z0001    0
X0101    0
Y0101    0
        ..
Y6420    0
Z6420    0
X6520    0
Y6520    0
Z6520    0
Length: 3960, dtype: int64

In [145]:
# X=np.array(X)
# X.shape
type(data.iloc[0])

pandas.core.series.Series

SVC

In [146]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
x_train, x_test, y_train, y_test = train_test_split(data, Y, test_size=0.2, random_state=42)

In [147]:
x_train.head

<bound method NDFrame.head of           X0001       Y0001     Z0001       X0101       Y0101     Z0101  \
456    0.235029   11.942410  0.639900    0.431633   12.184688  0.616264   
6     -1.136776 -104.591851  1.221791   -1.136776 -104.591851  1.221791   
362   -0.037427   46.847488  1.278112    0.118165   48.186779  1.224513   
594   -1.152971   27.921926 -1.046066   -1.152971   27.921926 -1.046066   
439   -1.001243 -210.468926  7.456261   -1.001243 -210.468926  7.456261   
..          ...         ...       ...         ...         ...       ...   
71  -121.353164   40.527615  0.640427 -121.353164   40.527615  0.640427   
106   -1.574159  -65.980731  5.376639   -1.574159  -65.980731  5.376639   
270   -1.606777 -179.820228  9.252626   -1.606777 -179.820228  9.252626   
435   -1.007977  866.301058  6.319335   -1.007977  866.301058  6.319335   
102   -1.551976 -133.040594  4.154679   -1.551976 -133.040594  4.154679   

          X0201       Y0201     Z0201       X0301  ...      Z6220    

In [148]:
from sklearn.svm import SVC
# model=SVC(kernel='rbf', C=100, gamma=0.001, random_state=42, verbose=True, max_iter=1000, decision_function_shape='ovo')
# model.fit(x_train, y_train)
# pred=model.predict(x_test)

In [149]:
from sklearn.metrics import classification_report, confusion_matrix
# # print(confusion_matrix(y_test, pred))
# print(classification_report(y_test, pred))

In [150]:
# from sklearn.model_selection import GridSearchCV
# param_grid={'C': [0.1,1,10,100,1000], 'gamma': [1,0.1,0.01,0.001,0.0001]}
# grid=GridSearchCV(SVC(), param_grid, verbose=5)

In [151]:
# grid.fit(x_train, y_train)

In [152]:
# grid.best_params_

In [153]:
# grid_pred=grid.predict(x_test)

In [154]:
# a=classification_report(y_test, grid_pred)
# print(a)

Random Forest

In [155]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
# model.fit(x_train, y_train)
model.fit(x_train, y_train)

# Make predictions on the test set
pred = model.predict(x_test)

In [156]:
print(confusion_matrix(y_test, pred))

[[ 9  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 16  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  8  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  7  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  6  0  0  0  0  2  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  9  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  9  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 10  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  5  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 11  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  7  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  3  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  7  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  4  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  8  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  8  0]
 [ 0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0 14]]


In [157]:
a=classification_report(y_test, pred)
print(a)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         9
           1       1.00      0.94      0.97        17
           2       1.00      1.00      1.00         8
           3       1.00      1.00      1.00         7
           4       1.00      0.75      0.86         8
           5       1.00      1.00      1.00         9
           6       0.90      1.00      0.95         9
           7       0.91      1.00      0.95        10
           8       1.00      1.00      1.00         5
           9       0.85      1.00      0.92        11
          10       1.00      1.00      1.00         7
          11       1.00      1.00      1.00         3
          12       1.00      1.00      1.00         7
          13       1.00      1.00      1.00         4
          14       1.00      1.00      1.00         8
          15       1.00      1.00      1.00         8
          16       1.00      0.93      0.97        15

    accuracy              

In [158]:
import pickle

with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)